# Predicting FWI using deepFWI Pre-trained models

### Introduction
The Fire Weather Index (FWI) is a meteorologically based index used worldwide to estimate fire danger. It consists of different components that account for the effects of fuel moisture and wind on fire behaviour and spread. The higher the FWI, the more favorable the meteorological conditions to trigger a wildfire. This indicator can help shape long-term tourist strategy and to plan future investments under a changing climate.

The [`esowc/wildfire-forecasting`](https://github.com/esowc/wildfire-forecasting) project intends to reproduce the Fire Forecasting capabilities of GEFF using Deep Learning and develop further improvements in accuracy, geography and time scale through inclusion of additional variables or optimisation of model architecture & hyperparameters. 

In this notebook we demonstrate the use of pre-trained Deep Learning models to perform FWI inference. The Deep Learning Model has been trained on 1 year of data and takes as input the four weather forcings used in the numerical calculation of FWI:
```
* t2     -    Temperature at 2m (K)
* tp     -    Total Precipitation accumulated over the previous 24 hours (mm)
* rh     -    Relative Humidity (%)
* wspeed -    Windspeed (m/s)
```

These input weather forcings are supplied for 4 days and the model produces FWI forecast for 10 days. There is 1 day of overlap between the 4 weather forcing variable inputs and the output FWI forecast as indicated below:
```
                                       ____________
Weather Forcings (t-3, t-2, t-1, t) -> | deepFWI  |  -> deepFWI-Forecast (t, t+1, t+2, ..., t+13)
                                       ------------
```

Along with FWI inference using deepGEFF pre-trained models, we also benchmark these results against the baseline `FWI-Forecast` which are numerically calculated FWI predictions from weather forecast.

To run this notebook, make sure you have the `wildfire-dl` `conda` environment activated. Instructions for setting up the environment are available in the project [`README`](https://github.com/esowc/wildfire-forecasting/blob/master/README.md).

## Getting the data

We first obtain a sample dataset. This is the minimum data that we need to perform inference and then benchmark the results. This data is stored on a publicly accessible Google Cloud Platform (GCP) storage bucket at `gs://deepgeff-data-v0/` and is downloaded into `sampledata/`

In [1]:
import os
os.chdir('..')

The `gs://deepfwi-mini-sample` dataset contains the following data - 
* `fwi-forcings/`: Weather Forcings for 2019-12-01 to 2019-12-04 --> 4 days --> 4 files 
* `fwi-forecast/`: Numerical FWI Predictions for 2019-12-04 to 2019-12-13 --> 10 days --> 1 file
* `fwi-reanalysis/`: ERA Reanalysis of FWI for 2019-12-04 to 2019-12-13 --> 10 days --> 10 files


If you wish to explore these datasets further, have a look at the EDA notebooks in `data/EDA/` in the repository.

In [2]:
# Uncomment to download a minimal sample dataset consisting of Weather Forcings
# for 2019-12-01 to 2019-12-04 --> 4 days --> 4 files
!mkdir sampledata/4_14/forcings -p
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/forcings/ECMWF_FO_2019120[1-4]*' sampledata/4_14/forcings

In [3]:
# Uncomment to download a minimal sample dataset consisting of ERA Reanalysis 
# for 2019-12-04 to 2019-12-17 --> 14 days --> 14 files
!mkdir sampledata/4_14/fwi-reanalysis -p
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/fwi-reanalysis/ECMWF_FWI_201912[0][4-9]*' sampledata/4_14/fwi-reanalysis
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/fwi-reanalysis/ECMWF_FWI_201912[1][0-7]*' sampledata/4_14/fwi-reanalysis

In [4]:
# Uncomment to download a minimal sample dataset consisting of Numerical FWI Predictions
# for 2019-12-04 to 2019-12-13 --> 10 days --> 1 file
!mkdir sampledata/4_14/fwi-forecast -p
!gsutil -q cp 'gs://deepgeff-data-v0/fwi-forecast/ECMWF_FWI_20191204_1200_hr_fwi.nc' sampledata/4_14/fwi-forecast

## Getting the pre-trained model
For this example notebook, we use the following model-
* Run ID: 9x14d9sv that has been trained for 100 epochs on 12 months of T2, TP, RH & WSpeed data for April 2019 - March, 2020; with Undersampling and Class-Balanced loss.

If you wish to explore other models, have a look at [pre-trained_models.md](https://github.com/esowc/wildfire-forecasting/blob/master/src/model/checkpoints/pre-trained_models.md).

In [5]:
# Uncomment to download a sample pre-trained model checkpoint file
!mkdir samplemodel/4_14/ -p
!gsutil -q cp 'gs://deepgeff-models-v0/pre-trained_models/4_14/1ga5loax/1ga5loax_59_60.ckpt' samplemodel/4_14/ 

### Running the inference and the benchmark

For performing inference and then benchmarking the results, we use the `run(**kwargs)` function from [`src/test.py`](https://github.com/esowc/wildfire-forecasting/blob/master/src/test.py) which takes as argument a dict of configuration parameters and boolean for `benchmark`. You can check out more details about this function in [`test.py#L266`](https://github.com/esowc/wildfire-forecasting/blob/master/src/test.py#L266)

In [6]:
from src.test import run

Providing the configuration options,

```
* in_days        -    no of input days of weather forcings
* out_days       -    no of output days of fwi prediction
* checkpoint_file-    path to pretrained checkpoint file
* reanalysis_dir -    path to fwi-reanalysis data
* forcings_dir   -    path to fwi-forcings data
* forecast_dir   -    path to fwi-forecast data
```

In [9]:
config = dict(
    in_days=4,
    out_days=10,
    checkpoint_file="samplemodel/4_14/1ga5loax_59_60.ckpt",
    reanalysis_dir="sampledata/4_14/fwi-reanalysis",
    forcings_dir="sampledata/4_14/forcings",
    forecast_dir="sampledata/4_14/fwi-forecast",
)

### Results

We demonstrate below the metrics for deepFWI predicted values and benchmark them against the numerical fwi-forecast prediction values. Accuracy, Mean Squared Error (MSE) and Mean Absolute Error (MAE), being the best evaluation metrics for comparing regression models, these are presented for 1 FWI prediction (for 10 days) in a global setting. The metrics are calculated against fwi-reanalysis (considered the ground truth). The grouped bar plots indicate the metrics for deepFWI while the line plots are used for fwi-forecast. The FWI values are binned as per the EFFIS categorisation:

```
* [0, 5.2]     - Very Low
* (5.2, 11.2]  - Low
* (11.2, 21.3] - Moderate
* (21.3, 38.0] - High
* (38.0, 50.0] - Very High
* (50.0, Inf)  - Extreme
```

As seen, the deepFWI predictions are consistent with FWI-Forecast and even offer better accuracy for longer term predictions in certain FWI categories. 

In [10]:
run(**config);

Doing inference with the model..


RuntimeError: Error(s) in loading state_dict for Model:
	size mismatch for conv.weight: copying a param with shape torch.Size([14, 16, 1, 1]) from checkpoint, the shape in current model is torch.Size([10, 16, 1, 1]).
	size mismatch for conv.bias: copying a param with shape torch.Size([14]) from checkpoint, the shape in current model is torch.Size([10]).